<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GWP_case_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colab-env -q
!pip install google-cloud-aiplatform --upgrade -q


## CASE 0

In [ ]:
from typing import Optional, List
import os
from google.colab import userdata
from google import genai
from google.genai import types  # Import types for Tool and GenerateContentConfig


def display_cities(cities: list[str], preferences: Optional[str] = None):
    """Provides a list of cities based on the user's search query and preferences.

    Args:
        preferences (str): The user's preferences for the search, like skiing,
            beach, restaurants, bbq, etc.
        cities (list[str]): The list of cities being recommended to the user.

    Returns:
        list[str]: The list of cities being recommended to the user.
    """
    return cities


# Define the function declaration for the model
display_cities_function = {
    "name": "display_cities",
    "description": "Provides a list of cities based on the user's search query and preferences.",
    "parameters": {
        "type": "object",
        "properties": {
            "cities": {
                "type": "array",
                "items": {"type": "string"},
                "description": "List of cities being recommended to the user.",
            },
            "preferences": {
                "type": "string",
                "description": "The user's preferences for the search, like skiing, beach, restaurants, bbq, etc.",
            },
        },
        "required": ["cities"],  # Only 'cities' is required
    },
}


# Configure the client and tools
GOOGLE_API_KEY = userdata.get('GEMINI')
client = genai.Client(api_key=GOOGLE_API_KEY)  # Initialize the client with the API key

tools = types.Tool(function_declarations=[display_cities_function])  # Create Tool object with types.Tool
config = types.GenerateContentConfig(tools=[tools])  # Create GenerateContentConfig with types.GenerateContentConfig

# Send request with function declarations
response = client.models.generate_content(
    model="gemini-1.5-pro-latest",  # Specify the model
    contents="I'd like to take a ski trip with my family but I'm not sure where to go.",
    config=config,
)

# Check for a function call
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    # In a real app, you would call your function here:
    # result = display_cities(**function_call.args)
else:
    print("No function call found in the response.")

Function to call: display_cities
Arguments: {'cities': ['Aspen', 'Park City', 'Stowe', 'Whistler'], 'preferences': 'skiing'}


## CASE 1a and 1b

In [ ]:
from typing import Optional, List
import os
from google.colab import userdata
from google import genai
from google.genai import types

# 1. Define Flight Planning Functions
def find_flights(
    departure_city: str,
    arrival_city: str,
    departure_date: str,  # "YYYY-MM-DD"
    return_date: Optional[str] = None,
    cabin_class: Optional[str] = "economy",
    num_passengers: int = 1,
):
    """
    Finds available flights based on the specified criteria.
    """
    # In a real application, this would call a flight booking API
    return f"Simulated flight results for {departure_city} to {arrival_city}"


def book_flight(
    flight_id: str,
    passenger_names: List[str],
    contact_email: str,
    billing_address: str,
):
    """
    Books a flight for the given passengers.
    """
    # In a real application, this would call a flight booking API
    return "Simulated flight booking confirmation"


# 2. Define Function Declarations for the Model
find_flights_function = {
    "name": "find_flights",
    "description": "Finds available flights based on the specified criteria.",
    "parameters": {
        "type": "object",
        "properties": {
            "departure_city": {
                "type": "string",
                "description": "The city of departure (e.g., 'New York').",
            },
            "arrival_city": {
                "type": "string",
                "description": "The city of arrival (e.g., 'London').",
            },
            "departure_date": {
                "type": "string",
                "description": "The departure date (YYYY-MM-DD).",
            },
            "return_date": {
                "type": "string",
                "description": "The return date (YYYY-MM-DD). Optional.",
            },
            "cabin_class": {
                "type": "string",
                "description": "The cabin class (e.g., 'economy', 'business', 'first'). Optional, default is 'economy'.",
            },
            "num_passengers": {
                "type": "integer",
                "description": "The number of passengers. Optional, default is 1.",
            },
        },
        "required": ["departure_city", "arrival_city", "departure_date"],
    },
}

book_flight_function = {
    "name": "book_flight",
    "description": "Books a flight for the given passengers.",
    "parameters": {
        "type": "object",
        "properties": {
            "flight_id": {
                "type": "string",
                "description": "The ID of the flight to book.",
            },
            "passenger_names": {
                "type": "array",
                "items": {"type": "string"},
                "description": "A list of passenger names.",
            },
            "contact_email": {
                "type": "string",
                "description": "The contact email for the booking.",
            },
            "billing_address": {
                "type": "string",
                "description": "The billing address for the booking.",
            },
        },
        "required": ["flight_id", "passenger_names", "contact_email", "billing_address"],
    },
}

# 3. Configure the Client and Tools
GOOGLE_API_KEY = userdata.get('GEMINI')  # Get your API key
client = genai.Client(api_key=GOOGLE_API_KEY)

tools = types.Tool(function_declarations=[find_flights_function, book_flight_function])  # Add both functions!
config = types.GenerateContentConfig(tools=[tools])

# 4. Send Request (Example: Flight Search)
response = client.models.generate_content(
    model="gemini-1.5-pro-latest",
    contents="Find me flights from New York to Los Angeles on 2024-12-24.",
    config=config,
)

#print(response.candidates[0].content)


response2 = client.models.generate_content(
    model="gemini-1.5-pro-latest",
    contents="Book flight with ID 'XYZ123' for John Doe and Jane Doe, email is 'contact@example.com', billing is '123 Main St'.",
    config=config,
)

#print(response2.candidates[0].content)

print('\n')
# 5. Handle the Response
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    # Call your flight planning functions here!
    if function_call.name == "find_flights":
        results = find_flights(**function_call.args)
        print(f'Result: {results}')
    elif function_call.name == "book_flight":
        confirmation = book_flight(**function_call.args)
        print(confirmation)
else:
    print("No function call found in the response.")


# Example 2: Booking a Flight (Hypothetical - requires a flight_id)
response = client.models.generate_content(
     model="gemini-1.5-pro-latest",
     contents="Book flight with ID 'XYZ123' for John Doe and Jane Doe, email is 'contact@example.com', billing is '123 Main St'.",
     config=config,
 )

print('\n\n')
if response2.candidates[0].content.parts[0].function_call:
    function_call = response2.candidates[0].content.parts[0].function_call
    print(f"Function to call: {function_call.name}")
    print(f"Arguments: {function_call.args}")
    # Call your flight planning functions here!
    if function_call.name == "find_flights":
        results = find_flights(**function_call.args)
        print(f'Result: {results}')
    elif function_call.name == "book_flight":
        confirmation = book_flight(**function_call.args)
        print(confirmation)
else:
    print("No function call found in the response.")



Function to call: find_flights
Arguments: {'departure_date': '2024-12-24', 'departure_city': 'New York', 'arrival_city': 'Los Angeles'}
Result: Simulated flight results for New York to Los Angeles



Function to call: book_flight
Arguments: {'billing_address': '123 Main St', 'contact_email': 'contact@example.com', 'flight_id': 'XYZ123', 'passenger_names': ['John Doe', 'Jane Doe']}
Simulated flight booking confirmation


## CASE 2

LangChain and LangGraph Agent: The code demonstrating how to build an agent using the LangChain and LangGraph libraries.



In [ ]:
!pip install langgraph -q
!pip install langchain-google-genai -q
!pip install google-cloud-aiplatform --upgrade -q
!pip install python-dateutil
!pip install langchain -q

In [ ]:
import langchain

print(langchain.__version__)

0.3.24


In [ ]:
from typing import Optional, List, Dict, Any # Import Dict and Any
import os
from google.colab import userdata  # Or your secrets management
from langchain_core.tools import tool
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI  # For Gemini
from google import genai
from google.genai import types
import json  # For safer JSON parsing
#from functools import partial # Removed partial import
from warnings import filterwarnings
filterwarnings('ignore')


# 1. Define Tools (Simulated Flight Search)
@tool
def find_flights(
    departure_city: str,
    arrival_city: str,
    departure_date: str,  # "YYYY-MM-DD"
    return_date: Optional[str] = None,
    cabin_class: Optional[str] = "economy",
    num_passengers: int = 1,
):
    """
    Finds available flights based on the specified criteria.
    """
    # In a real application, this would call a flight booking API
    return f"Simulated flight results for {departure_city} to {arrival_city} on {departure_date}"


# 2. Define the Agent's State
class AgentState(BaseModel):
    user_query: str
    chat_history: List[tuple[str, str]] = Field(default_factory=list)
    flight_results: Optional[str] = None  # Store flight results
    # Add a field to store the agent
    #agent: dict = Field(default_factory=dict) # Removed agent field from AgentState

    flight_info: Dict[str, Any] = Field(default_factory=dict)  # Store flight details for follow-up
    intermediate_response: Optional[str] = None  # Store intermediate response for follow-up




# 3. Define the Gemini 2.0 Agent
def create_agent(tools: list, model: str = "gemini-1.5-pro-latest"):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful AI travel agent."),
            ("user", "{user_query}"),  # User query directly in the prompt
        ]
    )

    # Use Pydantic to structure the output
    class FlightSearchOutput(BaseModel):
        departure_city: str = Field(description="City of departure")
        arrival_city: str = Field(description="City of arrival")
        departure_date: str = Field(description="Departure date (YYYY-MM-DD)")
        return_date: Optional[str] = Field(
            description="Return date (YYYY-MM-DD), if any"
        )

    output_parser = PydanticOutputParser(pydantic_object=FlightSearchOutput)

    llm = ChatGoogleGenerativeAI(model=model)

    def parse_tool_output(response):
        """
        Safely parses the tool output from the LLM.
        This is crucial for handling variations in LangChain versions.
        """
        try:
            # Try to use Pydantic output parsing (more robust)
            return output_parser.parse(response.content)
        except Exception:
            # Fallback: Simple text extraction (less robust)
            return str(response.content)  # Or more sophisticated regex

    # Chain for tool use
    tool_chain = prompt | llm.bind_tools(tools) | parse_tool_output

    # Chain for generating the final response
    response_chain = prompt | llm

    return {"tool_chain": tool_chain, "response_chain": response_chain}

def agent_node(state: AgentState, config):
       agent = create_agent(tools=[find_flights])  # Assuming create_agent is defined

       if not state.flight_results:
           result = agent["tool_chain"].invoke(state)  # LLM interaction
           # Check if the response is a question about the return date
           if isinstance(result, str) and "When do you want to return?" in result:
               state.intermediate_response = result  # Store intermediate response
               # Update chat_history with the intermediate response
               state.chat_history.append((state.user_query, result))
               return {"response": result}  # LLM's question

           # If not a question, and it's a string, assume it's the final answer
           elif isinstance(result, str):
               return {"flight_results": result}  # LLM's flight results

           # Otherwise, assume it's a tool call
           else:
               # Update flight_info with parsed result (if it's a dictionary)
               if isinstance(result, dict):
                   state.flight_info.update(result)

               # If result is not a dictionary (e.g., string), it might be the flight results
               # So we store it in flight_results
               else:
                   return {"flight_results": str(result)}

       else:
           result = agent["response_chain"].invoke(state)  # LLM interaction (final)
           return {"response": result}  # LLM's final answer

# 4. Define Agent Logic and Graph
# Modify agent_node to accept only state
def agent_node0(state: AgentState, config): # Add config as a parameter
    # Access agent from the state
    #agent = state.agent # Removed accessing agent from state
    agent = create_agent(tools=[find_flights]) # Create agent inside agent_node






    if not state.flight_results:
        result = agent["tool_chain"].invoke(state)
        return {"flight_results": result}  # Store tool output
    else:
        result = agent["response_chain"].invoke(state)
        return {"response": result}


def should_continue(state: AgentState):
    # Return "continue" or "exit" instead of True/False
    return "continue" if not state.flight_results else "exit"


# Create the graph
graph_builder = StateGraph(AgentState)  # Initialize StateGraph directly


# Renamed the node to "agent_node"
graph_builder.add_node("agent_node", agent_node)  # Pass the agent_node function directly
graph_builder.set_entry_point("agent_node") # Update the entry point
# Update conditional edges to use "continue" and "exit" keys
graph_builder.add_conditional_edges(
    "agent_node", should_continue, {"continue": "agent_node", "exit": END}
)
graph = graph_builder.compile()


# 5. Run the Agent
GOOGLE_API_KEY = userdata.get("GEMINI")  # Get your API key
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
client = genai.Client(api_key=GOOGLE_API_KEY)

In [ ]:
inputs1 = {"user_query": "Find me flights from New York to Los Angeles next week.", "chat_history": []}
final_state1 = graph.invoke(inputs1)
print("State after initial query:", final_state1)

# Follow-up with return date
departure_date = "2024-12-20"
inputs2 = {
    "user_query": departure_date,  # User provides the specific departure date
    "chat_history": final_state1['chat_history'] + [(inputs1['user_query'], final_state1['flight_results'])],  # Update chat history
    "flight_results": None,  # Reset flight_results for the next interaction
    "intermediate_response": None  # Reset intermediate_response for the next interaction
}
#final_state2 = graph.invoke(inputs2)
final_state2 = graph.invoke(inputs2, {"recursion_limit": 50})  # Increase to a higher number
print("\nState after follow-up:", final_state2)

State after initial query: {'user_query': 'Find me flights from New York to Los Angeles next week.', 'chat_history': [], 'flight_results': 'When next week would you like to leave?'}

State after follow-up: {'user_query': '2024-12-20', 'chat_history': [('Find me flights from New York to Los Angeles next week.', 'When next week would you like to leave?')], 'flight_results': 'Do you have a return date?', 'intermediate_response': None}


In [ ]:
final_state1

In [ ]:
final_state2